In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Input, Dense, Lambda, Multiply, Reshape, Flatten, Masking, LSTM, Concatenate
from tensorflow.keras.models import Model

In [5]:
from gym.spaces import Discrete
from gym.spaces import Tuple
from gym.spaces.box import Box

In [6]:
throttle_brake = Discrete(3)  # -1 brake, 0 keep, 1 throttle
steering = Discrete(3)
action_space = Tuple([throttle_brake,steering])

N = 4 # FIXME not hard code
F = 6 # FIXME not hard code
state_space = Box(low=-np.inf, high=np.inf, shape=(N,F), dtype=np.float32)

In [ ]:
state_input = Input(shape=(N,F),name="state_in")

x = Dense(32,activation="relu",name="encoding_1")(state_input)
# x = Dense(32,activation="relu",name="encoding_2")(x)

x = 